In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.core import Var

M = AbstractModel()
opt = SolverFactory('glpk')

In [2]:
# Sets
M.Box = Set()
M.Type = Set()

In [3]:
# Paramters
M.BoxLimits = Param(M.Box, M.Type, within=NonNegativeIntegers)
M.BoxCapacity = Param(M.Box, within=NonNegativeIntegers)
M.TotalTypes = Param(M.Type, within=NonNegativeIntegers)

In [4]:
# Variables
M.Packing = Var(M.Box, M.Type, within=NonNegativeIntegers)
# M.BoxUsed = Var(M.Box, within=Binary, initialize=0)

In [9]:
# Objective
def MaximizeUsedSpace(M):
    return sum(M.Packing[b,t] for b in M.Box for t in M.Type)
#     amount = 0
#     for b in M.Box:
#         capacity = M.BoxCapacity[b]
#         amount += sum(M.Packing[b,t]*M.BoxLimits[b,t]/capacity for t in M.Type)
#     return amount

# M.UsedSpace = Objective(rule=MaximizeUsedSpace, sense=maximize)

def MinimizeRemainingSpace(M):
    space_left = sum(M.BoxCapacity[b] for b in M.Box)
    for b in M.Box:
        space_left -= sum(M.Packing[b,t] for t in M.Type)
    return space_left
M.UsedSpace = Objective(rule=MinimizeRemainingSpace, sense=minimize)

	This is usually indicative of a modelling error.
	To avoid this warning, use block.del_component() and block.add_component().


In [6]:
# Constraints
def EnsureTotalTypes(M,t):
    return M.TotalTypes[t] == sum(M.Packing[b,t] for b in M.Box)
M.EnsureTotals = Constraint(M.Type, rule=EnsureTotalTypes)

def EnsureBoxTypeLimit(M,b,t):
    return M.Packing[b,t] <= M.BoxLimits[b,t]
M.EnsureBoxTypeLimit = Constraint(M.Box, M.Type, rule=EnsureBoxTypeLimit)

def EnsureCapacity(M,b):
    return sum(M.Packing[b,t] for t in M.Type) <= M.BoxCapacity[b]
M.EnsureCapacity = Constraint(M.Box, rule=EnsureCapacity)

# def ForceBoxUsed(M,b):
#     amount = sum(M.Packing[b,t] for t in M.Type)
#     if sum(M.Packing[b,t] for t in M.Type) == 0:
#         return M.BoxUsed[b] == 0
#     else:
#         return M.BoxUsed[b] == 1
# M.ForceBoxUsed = Constraint(M.Box, rule=ForceBoxUsed)

# def ForceBoxUnused(M,b):
#     return M.BoxUsed[b] == 0 and sum(M.Packing[b,t] for t in M.Type) == 0
# M.ForceBoxUnused = Constraint(M.Box, rule=ForceBoxUnused)

In [7]:
instance = M.create_instance('data/containers.dat')
results = opt.solve(instance, tee=True)
results.write()
instance.solutions.load_from(results)

GLPSOL: GLPK LP/MIP Solver, v4.57
Parameter(s) specified in the command line:
 --write /var/folders/fn/qz4yf0fx109ck2tsf5l__43c0000gn/T/tmptdayr469.glpk.raw
 --wglp /var/folders/fn/qz4yf0fx109ck2tsf5l__43c0000gn/T/tmpmgoxo_yx.glpk.glp
 --cpxlp /var/folders/fn/qz4yf0fx109ck2tsf5l__43c0000gn/T/tmpxrdwkmlf.pyomo.lp
Reading problem data from '/var/folders/fn/qz4yf0fx109ck2tsf5l__43c0000gn/T/tmpxrdwkmlf.pyomo.lp'...
15 rows, 9 columns, 25 non-zeros
8 integer variables, none of which are binary
103 lines were read
Writing problem data to '/var/folders/fn/qz4yf0fx109ck2tsf5l__43c0000gn/T/tmpmgoxo_yx.glpk.glp'...
84 lines were written
GLPK Integer Optimizer, v4.57
15 rows, 9 columns, 25 non-zeros
8 integer variables, none of which are binary
Preprocessing...
6 rows, 8 columns, 16 non-zeros
8 integer variables, one of which is binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangula

In [8]:
for v in instance.component_objects(Var, active=True):
    print ("Variable",v)
    varobject = getattr(instance, str(v))
    for index in varobject:
        print ("   ",index, varobject[index].value)

Variable Packing
    ('box2', 'Normal') 1.0
    ('box2', 'Rare') 3.0
    ('box4', 'Normal') 3.0
    ('box4', 'Rare') 1.0
    ('box3', 'Normal') 0.0
    ('box3', 'Rare') 0.0
    ('box1', 'Normal') 4.0
    ('box1', 'Rare') 1.0
